#### 1- Cloning the code2vec repository

In [ ]:
cd code2vec

In [ ]:
# Cannot be further trained ("released"):

!wget https://s3.amazonaws.com/code2vec/model/java14m_model.tar.gz

In [ ]:
!tar -xvzf java14m_model.tar.gz

In [ ]:
# Trainable model:

!wget https://s3.amazonaws.com/code2vec/model/java14m_model_trainable.tar.gz

In [ ]:
!tar -xvzf java14m_model_trainable.tar.gz #

#### 2- Preprocessing the dataset

In [ ]:
# Preprocessing on Methods in input(methods).java:

import progressbar

number_of_samples = 5000

c = 0

with open('/content/.../train/input.methods.txt','r') as dataset_input:
    with open('/content/.../train/output.tests.txt','r') as dataset_output:
        with open('input(methods).java','w') as input_file:
            with open('output(tests).java','w') as output_file:
                temp_output = dataset_output.readlines()
                for i,line in enumerate(dataset_input):
                    if i == number_of_samples:
                        break
                    if line.endswith('\t'):
                        c = c + 1
                        continue
                    if not line.endswith('}\n'):
                        c = c + 1
                        continue
                    if line.endswith('{ }\n'):
                        c = c + 1
                        continue
                    if line.endswith('{}\n'):
                        c = c + 1
                        continue
                    if line.find('IterateOverProperties') > -1:
                        c = c + 1
                        continue
                    if line.find('MockUp') > -1:
                        c = c + 1
                        continue
                    if line.find(' Pattern() ') > -1:
                        c = c + 1
                        continue
                    if line.find('forSourceLines') > -1:
                        c = c + 1
                        continue
                    if line.find('AdvertiserBlocklistRequester') > -1:
                        c = c + 1
                        continue
                    if line.find('addPartTwiceShouldCloseOtherPartTest') > -1:
                        c = c + 1
                        continue

                    count_public = line.count("public ")
                    loc_public = line.find("public ")
                    count_private = line.count("private ")
                    loc_private = line.find("private ")
                    count_protected = line.count("protected ")
                    loc_protected = line.find("protected ")
                    sum = count_public + count_private + count_protected

                    if line.startswith('static'):
                        if sum == 1:
                            if (not line.startswith('static public') and count_public) or (not line.startswith('static private') and count_private) or (not line.startswith('static protected') and count_protected):
                                c = c + 1
                                continue

                    if (sum == 1 and (loc_public > 120 or loc_private > 120 or loc_protected > 120)):
                        #print(line)
                        c = c + 1
                        continue

                    if sum < 2:
                        input_file.write(line)
                        output_file.write(temp_output[i])
                    else:
                        c = c + 1

remained_number_of_samples = number_of_samples - c

print('===================================')
print("Total Samples that removed = ",c)
print('===================================')
print("Total Samples that remain  = ", (remained_number_of_samples))
print('===================================')

In [ ]:
import subprocess


with open("input(methods).java", "r", encoding='utf-8') as f:
    lines = f.readlines()

with open("input(methods)_TEMP.java", "w", encoding='utf-8') as f:
    for line in lines:
        f.write(line)


path = 'input(methods)_TEMP.java'
max_path_length = 8
max_path_width = 2
jar_path = 'JavaExtractor/JPredict/target/JavaExtractor-0.0.1-SNAPSHOT.jar'


error_indexes = []
flag_samples_remain = True

redundant_method_paths = []

while remained_number_of_samples:
    command = ['java', '-cp', jar_path, 'JavaExtractor.App', '--max_path_length',
                    str(max_path_length), '--max_path_width', str(max_path_width), '--file', path, '--no_hash']

    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = process.communicate()
    output = out.decode().splitlines()

    #if len(output) > 1: #never happen
    #  redundant_method_paths.append(output)

    if len(output) == 0 and flag_samples_remain:
        err = err.decode()
        num = ""
        read_int = err.find("at line ") + 8

        while err[read_int] != ",":
            num = num + err[read_int]
            read_int = read_int + 1

        num = int(num)

        if error_indexes:
            error_indexes.append(num + error_indexes[-1])
        else:
            error_indexes.append(num - 1)

        print("Error Index = " , error_indexes[-1])

        with open("input(methods).java", "r", encoding='utf-8') as f:
            lines = f.readlines()

        with open("input(methods)_TEMP.java", "w", encoding='utf-8') as f:
            for i,line in enumerate(lines):
                if i > (error_indexes[-1]):
                    f.write(line)

        if error_indexes[-1] == (remained_number_of_samples - 1):
            flag_samples_remain = False

    else:
        remained_number_of_samples = remained_number_of_samples - len(error_indexes)
        print('===================================')
        print("Total Samples that removed = ",len(error_indexes))
        print('===================================')
        print("Total Samples that remain  = ",remained_number_of_samples)
        print('===================================')
        print("Total Redundant Vector     = ",len(redundant_method_paths))
        print('===================================')

        break

for i in range(len(error_indexes)):
   error_indexes[i] = error_indexes[i]


with open("input(methods).java", "r", encoding='utf-8') as f:
    lines = f.readlines()

with open("input(methods).java", "w", encoding='utf-8') as f:
    for i,line in enumerate(lines):
        if i not in error_indexes:
            f.write(line)


with open("output(tests).java", "r", encoding='utf-8') as f:
    lines = f.readlines()

with open("output(tests).java", "w", encoding='utf-8') as f:
    for i,line in enumerate(lines):
        if i not in error_indexes:
            f.write(line)

In [ ]:
# Preprocessing on Test Cases in output(tests).java:

number_of_samples = remained_number_of_samples

c = 0

with open('/content/.../code2vec/input(methods).java','r') as dataset_input:
    with open('/content/.../code2vec/output(tests).java','r') as dataset_output:
        with open('input(methods)_Temp4Test.java','w') as input_file:
            with open('output(tests)_Temp4Test.java','w') as output_file:
                temp_input = dataset_input.readlines()
                for i,line in enumerate(dataset_output):
                    if i == number_of_samples:
                        break
                    if line.endswith('\t'):
                        c = c + 1
                        continue
                    if not line.endswith('}\n'):
                        c = c + 1
                        continue
                    if line.endswith('{ }\n'):
                        c = c + 1
                        continue
                    if line.endswith('{}\n'):
                        c = c + 1
                        continue
                    if line.find('IterateOverProperties') > -1:
                        c = c + 1
                        continue
                    if line.find('MockUp') > -1:
                        c = c + 1
                        continue
                    if line.find(' Pattern() ') > -1:
                        c = c + 1
                        continue
                    if line.find('forSourceLines') > -1:
                        c = c + 1
                        continue
                    if line.find('AdvertiserBlocklistRequester') > -1:
                        c = c + 1
                        continue
                    if line.find('addPartTwiceShouldCloseOtherPartTest') > -1:
                        c = c + 1
                        continue

                    count_public = line.count("public ")
                    loc_public = line.find("public ")
                    count_private = line.count("private ")
                    loc_private = line.find("private ")
                    count_protected = line.count("protected ")
                    loc_protected = line.find("protected ")
                    sum = count_public + count_private + count_protected

                    if line.startswith('static'):
                        if sum == 1:
                            if (not line.startswith('static public') and count_public) or (not line.startswith('static private') and count_private) or (not line.startswith('static protected') and count_protected):
                                c = c + 1
                                continue

                    if (sum == 1 and (loc_public > 120 or loc_private > 120 or loc_protected > 120)):
                        #print(line)
                        c = c + 1
                        continue

                    if sum < 2:
                        output_file.write(line)
                        input_file.write(temp_input[i])
                    else:
                        c = c + 1

remained_number_of_samples = number_of_samples - c

print('===================================')
print("Total Samples that removed = ",c)
print('===================================')
print("Total Samples that remain  = ", (remained_number_of_samples))
print('===================================')

In [ ]:
import subprocess

with open("output(tests)_Temp4Test.java", "r", encoding='utf-8') as f:
    lines = f.readlines()

with open("output(tests)_TEMP.java", "w", encoding='utf-8') as f:
    for line in lines:
        f.write(line)


path = 'output(tests)_TEMP.java'
max_path_length = 8
max_path_width = 2
jar_path = 'JavaExtractor/JPredict/target/JavaExtractor-0.0.1-SNAPSHOT.jar'


error_indexes = []
flag_samples_remain = True

redundant_test_paths = []

while remained_number_of_samples:
    command = ['java', '-cp', jar_path, 'JavaExtractor.App', '--max_path_length',
                    str(max_path_length), '--max_path_width', str(max_path_width), '--file', path, '--no_hash']

    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = process.communicate()
    output = out.decode().splitlines()

    #if len(output) > 1: #Never happen
    #  redundant_test_paths.append(output)

    if len(output) == 0 and flag_samples_remain:
        #print(err)
        err = err.decode()
        num = ""
        read_int = err.find("at line ") + 8

        while err[read_int] != ",":
            num = num + err[read_int]
            read_int = read_int + 1

        num = int(num)

        if error_indexes:
            error_indexes.append(num + error_indexes[-1])
        else:
            error_indexes.append(num - 1)

        print("Error Index = " , error_indexes[-1])

        with open("output(tests)_Temp4Test.java", "r", encoding='utf-8') as f:
            lines = f.readlines()

        with open("output(tests)_TEMP.java", "w", encoding='utf-8') as f:
            for i,line in enumerate(lines):
                if i > (error_indexes[-1]):
                    f.write(line)

        if error_indexes[-1] == (remained_number_of_samples - 1):
            flag_samples_remain = False

    else:
        remained_number_of_samples = remained_number_of_samples - len(error_indexes)
        print('===================================')
        print("Total Samples that removed = ",len(error_indexes))
        print('===================================')
        print("Total Samples that remain = ",remained_number_of_samples)
        print('===================================')
        break

for i in range(len(error_indexes)):
   error_indexes[i] = error_indexes[i]


with open("output(tests)_Temp4Test.java", "r", encoding='utf-8') as f:
    lines = f.readlines()

with open("output(tests).java", "w", encoding='utf-8') as f:
    for i,line in enumerate(lines):
        if i not in error_indexes:
            f.write(line)


with open("input(methods)_Temp4Test.java", "r", encoding='utf-8') as f:
    lines = f.readlines()

with open("input(methods).java", "w", encoding='utf-8') as f:
    for i,line in enumerate(lines):
        if i not in error_indexes:
            f.write(line)

In [ ]:
# Saving cleaned Samples:

with open('input(methods).java','r', encoding='utf-8') as firstfile, open('input(methods)_5000.java','w', encoding='utf-8') as secondfile:
    for i,line in enumerate(firstfile):
        secondfile.write(line)

with open('output(tests).java','r', encoding='utf-8') as firstfile, open('output(tests)_5000.java','w', encoding='utf-8') as secondfile:
    for i,line in enumerate(firstfile):
        secondfile.write(line)

#### 3- Generating embedded vectors with pretrained model

In [ ]:
# Generating embedded vectors for Methods in input(methods).java:

with open('input(methods).java','r', encoding='utf-8') as firstfile, open('Input.java','w', encoding='utf-8') as secondfile:
    for i,line in enumerate(firstfile):
        secondfile.write(line)

In [ ]:
!rm code_vector_results.txt

In [ ]:
!rm high_attention_path.txt

In [ ]:
!python code2vec.py --load models/java14m/saved_model_iter8 --predict --export_code_vectors

In [ ]:
with open('Input.java','r', encoding='utf-8') as TempTest:
    print(TempTest.readlines()[0])

In [ ]:
with open('code_vector_results.txt','r') as firstfile, open('Dataset_Vectors_Methods.txt','w') as secondfile:
    for line in firstfile:
        secondfile.write(line)

In [ ]:
import pandas as pd

vectors_methods = pd.read_csv('Dataset_Vectors_Methods.txt', sep=" ", header=None)

In [ ]:
vectors_methods[vectors_methods.index == 0]

In [ ]:
vectors_methods.head()

In [ ]:
# Generating embedded vectors for Test Cases in output(tests).java:

with open('output(tests).java','r', encoding='utf-8') as firstfile, open('Input.java','w', encoding='utf-8') as secondfile:
    for i,line in enumerate(firstfile):
        secondfile.write(line)

In [ ]:
!rm code_vector_results.txt

In [ ]:
!python code2vec.py --load models/java14m/saved_model_iter8 --predict --export_code_vectors

In [ ]:
with open('code_vector_results.txt','r') as firstfile, open('Dataset_Vectors_Tests.txt','w') as secondfile:
    for line in firstfile:
        secondfile.write(line)

In [ ]:
import pandas as pd

vectors_tests = pd.read_csv('Dataset_Vectors_Tests.txt', sep=" ", header=None)

In [ ]:
vectors_tests.head()

### 4- Training the model

In [ ]:
# Creat a dataset with method and test:

with open('input(methods)_5000.java','r', encoding='utf-8') as firstfile, open('Dataset_Merged_M&T.java','w', encoding='utf-8') as datasetfile:
    for i,line in enumerate(firstfile):
        datasetfile.write(line)

with open('output(tests)_5000.java','r', encoding='utf-8') as firstfile, open('Dataset_Merged_M&T.java','a', encoding='utf-8') as datasetfile:
    for i,line in enumerate(firstfile):
        datasetfile.write(line)

In [ ]:
# Shuffling dataset raws:

import random

with open('Dataset_Merged_M&T.java','r', encoding='utf-8') as datasetfile:
    data = [ (random.random(), line) for line in datasetfile ]

data.sort()

with open('Dataset_Merged_M&T.java','w', encoding='utf-8') as datasetfile:
    for _,line in data:
        datasetfile.write(line)

In [ ]:
# Spliting the Dataset:

number_of_sample_in_dataset = len(data)
number_of_train = int(number_of_sample_in_dataset * 0.8)
number_of_test_and_val = int((number_of_sample_in_dataset - number_of_train)/2)

counter_train_test_val = 0

with open('Dataset_Merged_M&T.java','r', encoding='utf-8') as datasetfile:
    with open('Dataset_Merged_M&T_train.java','w', encoding='utf-8') as datasetfile_train:
        with open('Dataset_Merged_M&T_val.java','w', encoding='utf-8') as datasetfile_val:
            with open('Dataset_Merged_M&T_test.java','w', encoding='utf-8') as datasetfile_test:

                for line in datasetfile:
                    if (counter_train_test_val < number_of_train):
                        datasetfile_train.write(line)
                    elif (number_of_train <= counter_train_test_val and counter_train_test_val < (number_of_train + number_of_test_and_val)):
                        datasetfile_val.write(line)
                    else:
                        datasetfile_test.write(line)

                    counter_train_test_val = counter_train_test_val + 1


In [ ]:
!source preprocess.sh

In [ ]:
!source train.sh --load models/java14m_trainable/saved_model_iter8.release --data data/MyDataset --save

### 5- Loading and visualizing the generated embedded vectors

In [ ]:
# Loading vectors:

import pandas as pd
vectors_methods = pd.read_csv('Dataset_Vectors_Methods_5000.txt', sep=" ", header=None)
vectors_tests = pd.read_csv('Dataset_Vectors_Tests_5000.txt', sep=" ", header=None)


In [ ]:
# Loading cleaned samples:

with open('input(methods)_5000.java','r', encoding='utf-8') as firstfile, open('input(methods).java','w', encoding='utf-8') as secondfile:
    for i,line in enumerate(firstfile):
        secondfile.write(line)

with open('output(tests)_5000.java','r', encoding='utf-8') as firstfile, open('output(tests).java','w', encoding='utf-8') as secondfile:
    for i,line in enumerate(firstfile):
        secondfile.write(line)


In [ ]:
# Visualization of the Methods and Test Cases:

from sklearn.decomposition import PCA

pca = PCA(n_components = 2)

vectors_methods_2D = pca.fit_transform(vectors_methods)
vectors_tests_2D = pca.fit_transform(vectors_tests)

df_vectors_methods_2D = pd.DataFrame(vectors_methods_2D, columns = ['X1','X2'])
df_vectors_tests_2D = pd.DataFrame(vectors_tests_2D, columns = ['X1','X2'])

In [ ]:
print("Methods: ", df_vectors_methods_2D.shape)
print("Tests:   ", df_vectors_tests_2D.shape)

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()

def my_theme(*args, **kwargs):
    return {'config': {'background': '#303030',
                       'axis': {'domainColor': '#303030', 'tickColor':'black'},
                       'view': {'fill': '#303030', 'stroke':'#303030'},
                       'title': {'fontSize':16}}
            }

alt.themes.register('my-chart', my_theme)
alt.themes.enable('my-chart')



Chart_Methods = alt.Chart(df_vectors_methods_2D).mark_circle(
        color='red',
        opacity=0.3,
        size=60,
).encode(
    alt.X('X1', axis=alt.Axis(grid=False)),
    alt.Y('X2', axis=alt.Axis(grid=False)),
).interactive().properties(
    width=400,
    height=400,
    title='Methods'
)#.configure(background='#383838').configure_mark(color='darkyellow')


Chart_Tests = alt.Chart(df_vectors_tests_2D).mark_circle(
        color='green',
        opacity=0.3,
        size=60,
).encode(
    alt.X('X1', axis=alt.Axis(grid=False)),
    alt.Y('X2', axis=alt.Axis(grid=False)),
).interactive().properties(
    width=400,
    height=400,
    title='Test Cases'
)#.configure(background='#383838').configure_mark(color='darkyellow')

Chart_Methods | Chart_Tests

alt.HConcatChart(...)

### 6- Calculating the distances

In [ ]:
input_methods_f = open('input(methods).java', 'r')
methods = input_methods_f.readlines()
input_methods_f.close()

output_tests_f = open('output(tests).java', 'r')
tests = output_tests_f.readlines()
output_tests_f.close()

In [ ]:
print("Method:")
print(methods[0])
print("Test Case:")
print(tests[0])

Method:
@Override public boolean equals(Object obj) { if (this == obj) { return true; } if (obj == null) return false; if (getClass() != obj.getClass()) return false; SortedIntArray other = (SortedIntArray) obj; if (length != other.length) return false; for (int i = 0; i < length; i++) { if (a[i] != other.a[i]) return false; } return true; }

Test Case:
@Test public void testEquals() { SortedIntArray s1 = new SortedIntArray(1, 3, 7); SortedIntArray s2 = new SortedIntArray(3, 1, 7); SortedIntArray s3 = new SortedIntArray(1, 3); assertEquals(s1.hashCode(), s2.hashCode()); assertTrue(s1.equals(s2)); assertTrue(s2.equals(s1)); assertFalse(s1.equals(s3)); assertFalse(s3.equals(s1)); assertFalse(s1.hashCode() == s3.hashCode()); }



In [ ]:
# generating and saving distances_methods

'''
from scipy import spatial

distances_methods = []

for i in vectors_methods.index:
    distances_methods.append([])
    for j in range(i+1, len(vectors_methods)):
        distances_methods[i].append(round(1 - spatial.distance.cosine(vectors_methods.iloc[[i]], vectors_methods.iloc[[j]]),3))

#print('Size of methods distances matrix: (', len(distances_methods),',',len(distances_methods[0]),')')

with open('distances_methods.txt','w', encoding='utf-8') as f:
    for lst in distances_methods:
        f.write(str(lst) + '\n')

print("Done!")
'''

In [ ]:
# loading distances_methods (distances_methods_5000.txt)

import ast

with open('distances_methods.txt') as f:
    lines = f.read().splitlines()

distances_methods = []
for l in lines:
    distances_methods.append(ast.literal_eval(l))

del lines


In [ ]:
# generating and saving distances_tests

'''
from scipy import spatial

distances_tests = []

for i in vectors_tests.index:
    distances_tests.append([])
    for j in range(i+1, len(vectors_tests)):
        distances_tests[i].append(round(1 - spatial.distance.cosine(vectors_tests.iloc[[i]], vectors_tests.iloc[[j]]),3))

#print('Size of Test Cases distances matrix: (', len(distances_tests),',',len(distances_tests[0]),')')

with open('distances_tests.txt','w', encoding='utf-8') as f:
    for lst in distances_tests:
        f.write(str(lst) + '\n')

print("Done!")
'''

In [ ]:
# loading distances_tests (distances_tests_5000.txt)

import ast

with open('distances_tests.txt') as f:
    lines = f.read().splitlines()

distances_tests = []
for l in lines:
    distances_tests.append(ast.literal_eval(l))

del lines


In [ ]:
similarity_threshold = 0.90

In [ ]:
similar_methods_index = []
similar_methods_distance = []

for row, values in enumerate(distances_methods):
    for col in range(len(values)):
        if values[col] >= similarity_threshold :
            similar_methods_index.append((row,col+1+row))
            similar_methods_distance.append(distances_methods[row][col])

In [ ]:
print('There are (', len(similar_methods_index), ') Methods with more than (', similarity_threshold*100, '% ) similarity.')
print('\nlike: ', similar_methods_index[0], '=', similar_methods_distance[0])

In [ ]:
similar_tests_index = []
similar_tests_distance = []

for row, values in enumerate(distances_tests):
    for col in range(len(values)):
        if values[col] >= similarity_threshold :
            similar_tests_index.append((row,col+1+row))
            similar_tests_distance.append(distances_tests[row][col])

In [ ]:
print('There are (', len(similar_tests_index), ') Tests with more than (', similarity_threshold*100, '% ) similarity.')
print('\nlike: ', similar_tests_index[0], '=', similar_tests_distance[0])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


Labels = similar_methods_index
Values_method = similar_methods_distance

Values_test = []
for x in range(len(similar_methods_index)):
    i, j = similar_methods_index[x]
    dis = distances_tests[i][j-1-i]      #revers of (row,col+1+row)
    if dis > 0:
        Values_test.append(dis)
    else:
        Values_test.append(0)


angles=np.linspace(0,2*np.pi,len(Labels), endpoint=False)

angles=np.concatenate((angles,[angles[0]]))
Labels.append(Labels[0])
Values_method.append(Values_method[0])
Values_test.append(Values_test[0])

fig=plt.figure(figsize=(20,20),facecolor='#ffffff') #dadada
ax=fig.add_subplot(111, polar=True)

#First Plot
ax.plot(angles,Values_method, 'o-', color='green', linewidth=1, label='Methods')
ax.fill(angles, Values_method, alpha=0.25, color='green')

#Second Plot
ax.plot(angles,Values_test, 'o-', color='orange', linewidth=1, label='Test Cases')
ax.fill(angles, Values_test, alpha=0.25, color='orange')

#Labels

ax.set_thetagrids(angles * 180/np.pi, [])#, Labels)
ax.set_rlabel_position(0)


plt.yticks([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1], ['0.1','0.2','0.3','0.4','0.5','0.6','0.7','0.8','0.9'], size=10)
plt.ylim(0,1)

plt.grid(True)
plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
import statistics

In [ ]:
print("MM:")
print("Count:", len(Values_method))
print("Mean: ", statistics.mean(Values_method))
print("Std:  ", statistics.stdev(Values_method))
print("Max:  ", max(Values_method))
print("Min:  ", min(Values_method))

In [ ]:
print("TT:")
print("Count:", len(Values_test))
print("Mean: ", statistics.mean(Values_test))
print("Std:  ", statistics.stdev(Values_test))
print("Max:  ", max(Values_test))
print("Min:  ", min(Values_test))

In [ ]:
1 - (len([ x for x in Values_test if x > 0.5 ]) / len(Values_test))

In [ ]:
1 - (len([ x for x in Values_test if x > 0.7 ]) / len(Values_test))

In [ ]:
# libraries & dataset

import seaborn as sns
import matplotlib.pyplot as plt
# set a grey background (use sns.set_theme() if seaborn version 0.11.0 or above)
sns.set(style="darkgrid")
#df = sns.load_dataset("iris")

sns.histplot(data=Values_method, legend=False, color="Green", kde=True, bins=20)
plt.axvline(x=statistics.mean(Values_method),
            color='Green',
            ls='--',
            lw=1.75)

plt.legend()
plt.show()

sns.histplot(data=Values_test, legend=False, color="orange", kde=True, bins=20)
plt.axvline(x=statistics.mean(Values_test),
            color='orange',
            ls='--',
            lw=1.75)


In [ ]:
from scipy import spatial

distances_method_new = []

for i in vectors_methods.index:
    distances_method_new.append(round(1 - spatial.distance.cosine(vectors_methods.iloc[method_new], vectors_methods.iloc[i]),3))


In [ ]:
pip install textdistance

In [ ]:
import textdistance

similar_method_new_index = []
similar_method_new_distance = []
similar_test_new_distance = []
similar_test_new_levenshtein = []
similar_method_test_new_distance = []
similar_method_test_new_levenshtein = []

for i, dis in enumerate(distances_method_new):
    if i == method_new:
        continue
    if dis >= similarity_threshold :
            similar_method_new_index.append(i)
            similar_method_new_distance.append(dis)
            similar_test_new_distance.append(round(1 - spatial.distance.cosine(vectors_tests.iloc[method_new], vectors_tests.iloc[i]),3))
            similar_test_new_levenshtein.append(textdistance.levenshtein(tests[method_new], tests[i]))
            similar_method_test_new_distance.append(round(1 - spatial.distance.cosine(vectors_methods.iloc[method_new], vectors_tests.iloc[i]),3))
            similar_method_test_new_levenshtein.append(textdistance.levenshtein(methods[method_new], tests[i]))

similar_method_new = pd.DataFrame({'Index':similar_method_new_index,
                                   'Method_Distance':similar_method_new_distance,
                                   'Test_Distance':similar_test_new_distance,
                                   'Test_Levenshtein':similar_test_new_levenshtein,
                                   'Test_Method_Distance':similar_method_test_new_distance,
                                   'Test_Method_Levenshtein':similar_method_test_new_levenshtein})

similar_method_new.sort_values(by=['Test_Method_Levenshtein'], inplace=True, ignore_index=True)
display(similar_method_new)

In [ ]:
print('Sample', method_new,':')
print('============\n')

for i, M in enumerate(similar_method_new['Index']):
    #print('Sample', M, ':\n')
    #print('Cosine Similarity =', similar_method_new.iloc[i]['Method_Distance'], '):\n')
    #print(methods[M])
    print('Cosine Similarity =',       similar_method_new.iloc[i]['Test_Distance'])
    print('levenshtein\'s distance =', int(similar_method_new.iloc[i]['Test_Levenshtein']), '\n')
    print('Oracel (length =', len( tests[method_new]),'):         ', tests[method_new])
    print('Recommended (length =', len( tests[int(similar_method_new.iloc[i]['Index'])]),'):   ', tests[M])
    print('---------------------------------------------------------------------------\n')

In [ ]:
similar_methods_index_removed_duplicates = []

for i in range(len(similar_methods_index)):
    if similar_methods_index[i][0] not in similar_methods_index_removed_duplicates:
        similar_methods_index_removed_duplicates.append(similar_methods_index[i][0])


In [ ]:
# Generate all_similar_method_new:
'''
import textdistance
import pickle

all_similar_method_new = []

for i,M1 in enumerate(similar_methods_index_removed_duplicates):

    method_new = M1

    similar_method_new_index = []
    similar_method_new_distance = []
    similar_test_new_distance = []
    similar_test_new_levenshtein = []
    similar_method_test_new_distance = []
    similar_method_test_new_levenshtein = []


    Similar_M2_for_each_M1 = []
    for i,x in enumerate(similar_methods_index):
        if (x[0] is M1) and (x[1] not in Similar_M2_for_each_M1):
            Similar_M2_for_each_M1.append(x[1])

    for i, M2 in enumerate(Similar_M2_for_each_M1):
        similar_method_new_index.append(M2)
        similar_method_new_distance.append(round(1 - spatial.distance.cosine(vectors_methods.iloc[method_new], vectors_methods.iloc[M2]),3))
        similar_test_new_distance.append(round(1 - spatial.distance.cosine(vectors_tests.iloc[method_new], vectors_tests.iloc[M2]),3))
        similar_test_new_levenshtein.append(textdistance.levenshtein(tests[method_new], tests[M2]))
        similar_method_test_new_distance.append(round(1 - spatial.distance.cosine(vectors_methods.iloc[method_new], vectors_tests.iloc[M2]),3))
        similar_method_test_new_levenshtein.append(textdistance.levenshtein(methods[method_new], tests[M2]))

    similar_method_new = pd.DataFrame({'Index':similar_method_new_index,
                                      'Method_Distance':similar_method_new_distance,
                                      'Test_Distance':similar_test_new_distance,
                                      'Test_Levenshtein':similar_test_new_levenshtein,
                                      'Test_Method_Distance':similar_method_test_new_distance,
                                      'Test_Method_Levenshtein':similar_method_test_new_levenshtein})

    similar_method_new.sort_values(by=['Test_Method_Levenshtein'], inplace=True, ignore_index=True)

    all_similar_method_new.append(similar_method_new)

#display(similar_method_new)

print("Done!")
'''

In [ ]:
#Saving distances_methods_and_Test:
'''
import pickle

open_file = open("all_similar_method_new.pkl", "wb")
pickle.dump(all_similar_method_new, open_file)
open_file.close()
'''

In [ ]:
#Loading all_similar_method_new:

import pickle

open_file = open("all_similar_method_new.pkl", "rb")
all_similar_method_new = pickle.load(open_file)
open_file.close()


In [ ]:
all_similar_method_new[100]

In [ ]:
#saveing all_similar_method_new:
'''
import pickle

open_file = open("all_similar_method_new.pkl", "wb")
pickle.dump(all_similar_method_new, open_file)
open_file.close()
'''

### 7- First approach

In [ ]:
Analysis_similarity_Method_Label_App1 = []
Analysis_similarity_Method_Method_App1 = []
Analysis_similarity_Test_Test_App1 = []
Analysis_similarity_Method_Test_App1 = []

for i in range(len(all_similar_method_new)):
    all_similar_method_new[i].sort_values(by=['Test_Method_Distance'], ascending=False, inplace=True)
    Analysis_similarity_Method_Label_App1.append((int(similar_methods_index_removed_duplicates[i]),int(all_similar_method_new[i].iloc[0]['Index'])))
    Analysis_similarity_Method_Method_App1.append(all_similar_method_new[i].iloc[0]['Method_Distance'])
    Analysis_similarity_Test_Test_App1.append(all_similar_method_new[i].iloc[0]['Test_Distance'])
    Analysis_similarity_Method_Test_App1.append(all_similar_method_new[i].iloc[0]['Test_Method_Distance'])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


Labels = Analysis_similarity_Method_Label_App1
Values_method = Analysis_similarity_Method_Method_App1
Values_test = Analysis_similarity_Test_Test_App1


angles=np.linspace(0,2*np.pi,len(Labels), endpoint=False)

angles=np.concatenate((angles,[angles[0]]))
Labels.append(Labels[0])
Values_method.append(Values_method[0])
Values_test.append(Values_test[0])

fig=plt.figure(figsize=(10,10),facecolor='#ffffff') #dadada
ax=fig.add_subplot(111, polar=True)

#First Plot
ax.plot(angles,Values_method, 'o-', color='green', linewidth=1, label='Methods')
ax.fill(angles, Values_method, alpha=0.25, color='green')

#Second Plot
ax.plot(angles,Values_test, 'o-', color='orange', linewidth=1, label='Test Cases')
ax.fill(angles, Values_test, alpha=0.25, color='orange')

#Labels

ax.set_thetagrids(angles * 180/np.pi, []) #Labels)
ax.set_rlabel_position(0)


plt.yticks([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1], ['0.1','0.2','0.3','0.4','0.5','0.6','0.7','0.8','0.9'], size=10)
plt.ylim(0,1)

plt.grid(True)
plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
print("MM - Approach 1:")
print("Count:", len(Values_method))
print("Mean: ", statistics.mean(Values_method))
print("Std:  ", statistics.stdev(Values_method))
print("Max:  ", max(Values_method))
print("Min:  ", min(Values_method))

In [ ]:
print("TT - Approach 1:")
print("Count:", len(Values_test))
print("Mean: ", statistics.mean(Values_test))
print("Std:  ", statistics.stdev(Values_test))
print("Max:  ", max(Values_test))
print("Min:  ", min(Values_test))

In [ ]:
1 - (len([ x for x in Values_test if x > 0.5 ]) / len(Values_test))

In [ ]:
1 - (len([ x for x in Values_test if x > 0.7 ]) / len(Values_test))

In [ ]:
sns.set(style="darkgrid")

sns.set(rc={'figure.figsize':(6,4)})

sns.histplot(data=Values_test, legend=False, color="orange", kde=True, bins=20)
plt.axvline(x=statistics.mean(Values_test),
            color='orange',
            ls='--',
            lw=1.75)

plt.legend()
plt.show()

### 8- Second approach

In [ ]:
# generating and saving distances_methods_and_Test
'''
from scipy import spatial

distances_methods_and_Test = []

for i in vectors_methods.index:
    distances_methods_and_Test.append([])
    for j in vectors_tests.index:
        distances_methods_and_Test[i].append(round(1 - spatial.distance.cosine(vectors_methods.iloc[[i]], vectors_tests.iloc[[j]]),3))

print('Size of methods distances matrix: (', len(distances_methods_and_Test),',',len(distances_methods_and_Test[0]),')')

print("Done")
'''

In [ ]:
#Saving distances_methods_and_Test:
'''
import pickle

open_file = open("distances_methods_and_Test.pkl", "wb")
pickle.dump(distances_methods_and_Test, open_file)
open_file.close()
'''

In [ ]:
#Loading distances_methods_and_Test:
import pickle

open_file = open("distances_methods_and_Test.pkl", "rb")
distances_methods_and_Test = pickle.load(open_file)
open_file.close()

In [ ]:
print(len(distances_methods_and_Test) , 'x' , len(distances_methods_and_Test[0]))

In [ ]:
similarity_threshold_Method_and_Test = 0.70

similar_methods_and_Test_index = []
similar_methods_and_Test_distance = []

for row, values in enumerate(distances_methods_and_Test):
    for col in range(len(values)):
        if values[col] >= similarity_threshold_Method_and_Test :
            similar_methods_and_Test_index.append((row,col))
            similar_methods_and_Test_distance.append(distances_methods_and_Test[row][col])



In [ ]:
print('There are (', len(similar_methods_and_Test_index), ') Methods and Tests with more that (', similarity_threshold_Method_and_Test*100, '% ) similarity.')
print('\nlike: ', similar_methods_and_Test_index[0], '=', similar_methods_and_Test_distance[0])
print('\nlike: ', similar_methods_and_Test_index[3], '=', similar_methods_and_Test_distance[3])

In [ ]:
similar_methods_and_test_index_removed_duplicates = []

for i in range(len(similar_methods_and_Test_index)):
    if similar_methods_and_Test_index[i][0] not in similar_methods_and_test_index_removed_duplicates:
        similar_methods_and_test_index_removed_duplicates.append(similar_methods_and_Test_index[i][0])

In [ ]:
len(similar_methods_and_test_index_removed_duplicates)

In [ ]:
# Generate all_similar_tests_to_method_new:
'''
import textdistance
import pickle

all_similar_method_and_test_new = []

for i,M1 in enumerate(similar_methods_and_test_index_removed_duplicates):

    method_new = M1

    similar_tests_to_method_new_index = []
    similar_tests_to_method_new_distance = []
    similar_test_new_distance = []
    similar_test_new_levenshtein = []
    similar_method_method_new_distance = []
    similar_method_method_new_levenshtein = []


    Similar_T2_for_each_M1 = []
    for i,x in enumerate(similar_methods_and_Test_index):
        if (x[0] is M1) and (x[1] not in Similar_T2_for_each_M1):
            Similar_T2_for_each_M1.append(x[1])

    for i, M2 in enumerate(Similar_T2_for_each_M1):
        similar_tests_to_method_new_index.append(M2)
        similar_tests_to_method_new_distance.append(round(1 - spatial.distance.cosine(vectors_methods.iloc[method_new], vectors_tests.iloc[M2]),3))
        similar_test_new_distance.append(round(1 - spatial.distance.cosine(vectors_tests.iloc[method_new], vectors_tests.iloc[M2]),3))
        similar_test_new_levenshtein.append(textdistance.levenshtein(tests[method_new], tests[M2]))
        similar_method_method_new_distance.append(round(1 - spatial.distance.cosine(vectors_methods.iloc[method_new], vectors_methods.iloc[M2]),3))
        similar_method_method_new_levenshtein.append(textdistance.levenshtein(methods[method_new], methods[M2]))

    similar_tests_to_method_new = pd.DataFrame({'Index':similar_tests_to_method_new_index,
                                      'Method_and_Tests_Distance':similar_tests_to_method_new_distance,
                                      'Test_Distance':similar_test_new_distance,
                                      'Test_Levenshtein':similar_test_new_levenshtein,
                                      'Method_Method_Distance':similar_method_method_new_distance,
                                      'Method_Method_Levenshtein':similar_method_method_new_levenshtein})

    similar_tests_to_method_new.sort_values(by=['Method_Method_Distance'], inplace=True, ignore_index=True, ascending=False)

    all_similar_method_and_test_new.append(similar_tests_to_method_new)

#display(similar_tests_to_method_new)

print("Done!")
'''

In [ ]:
#saveing all_similar_method_and_Test_new:
'''
import pickle

open_file = open("all_similar_method_and_Test_new.pkl", "wb")
pickle.dump(all_similar_method_and_test_new, open_file)
open_file.close()
'''

In [ ]:
#Loading all_similar_method_new:
import pickle

open_file = open("all_similar_method_and_Test_new.pkl", "rb")
all_similar_method_and_test_new = pickle.load(open_file)
open_file.close()

In [ ]:
Analysis_similarity_Method_and_Test_Label_App2 = []
Analysis_similarity_Method_Test_App2 = []
Analysis_similarity_Test_Test_App2 = []

for i in range(len(all_similar_method_and_test_new)):
    #sorted befor based on "Method_Method_Distance"
    if (int(similar_methods_and_test_index_removed_duplicates[i]) != int(all_similar_method_and_test_new[i].iloc[0]['Index'])):
        Analysis_similarity_Method_and_Test_Label_App2.append((int(similar_methods_and_test_index_removed_duplicates[i]),int(all_similar_method_and_test_new[i].iloc[0]['Index'])))
        Analysis_similarity_Method_Test_App2.append(all_similar_method_and_test_new[i].iloc[0]['Method_and_Tests_Distance'])
        Analysis_similarity_Test_Test_App2.append(all_similar_method_and_test_new[i].iloc[0]['Test_Distance'])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


Labels = Analysis_similarity_Method_and_Test_Label_App2
Values_method = Analysis_similarity_Method_Test_App2
Values_test = Analysis_similarity_Test_Test_App2


angles=np.linspace(0,2*np.pi,len(Labels), endpoint=False)

angles=np.concatenate((angles,[angles[0]]))
Labels.append(Labels[0])
Values_method.append(Values_method[0])
Values_test.append(Values_test[0])

fig=plt.figure(figsize=(10,10),facecolor='#ffffff') #dadada
ax=fig.add_subplot(111, polar=True)

#First Plot
ax.plot(angles,Values_method, 'o-', color='green', linewidth=1, label='Methods')
ax.fill(angles, Values_method, alpha=0.25, color='green')

#Second Plot
ax.plot(angles,Values_test, 'o-', color='orange', linewidth=1, label='Test Cases')
ax.fill(angles, Values_test, alpha=0.25, color='orange')

#Labels

ax.set_thetagrids(angles * 180/np.pi, []) #Labels)
ax.set_rlabel_position(0)


plt.yticks([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1], ['0.1','0.2','0.3','0.4','0.5','0.6','0.7','0.8','0.9'], size=10)
plt.ylim(0,1)

plt.grid(True)
plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
print("MM - Approach 2:")
print("Count:", len(Values_method))
print("Mean: ", statistics.mean(Values_method))
print("Std:  ", statistics.stdev(Values_method))
print("Max:  ", max(Values_method))
print("Min:  ", min(Values_method))

In [ ]:
print("TT - Approach 2:")
print("Count:", len(Values_test))
print("Mean: ", statistics.mean(Values_test))
print("Std:  ", statistics.stdev(Values_test))
print("Max:  ", max(Values_test))
print("Min:  ", min(Values_test))

In [ ]:
1 - (len([ x for x in Values_test if x > 0.5 ]) / len(Values_test))

In [ ]:
1 - (len([ x for x in Values_test if x > 0.7 ]) / len(Values_test))

In [ ]:
sns.set(style="darkgrid")

sns.set(rc={'figure.figsize':(6,4)})

sns.histplot(data=Values_test, legend=False, color="orange", kde=True, bins=20)
plt.axvline(x=statistics.mean(Values_test),
            color='orange',
            ls='--',
            lw=1.75)

plt.legend()
plt.show()

### 9- First approach evaluation

In [ ]:
Analysis_similarity_Method_Label_App1_Levenshtein = []
Analysis_similarity_Method_Method_App1_Levenshtein = []
Analysis_similarity_Test_Test_App1_Levenshtein = []

for i in range(len(all_similar_method_new)):
    all_similar_method_new[i].sort_values(by=['Test_Method_Levenshtein'], ascending=True, inplace=True)
    Analysis_similarity_Method_Label_App1_Levenshtein.append((int(similar_methods_index_removed_duplicates[i]),int(all_similar_method_new[i].iloc[0]['Index'])))
    Analysis_similarity_Method_Method_App1_Levenshtein.append(all_similar_method_new[i].iloc[0]['Method_Distance'])
    Analysis_similarity_Test_Test_App1_Levenshtein.append(all_similar_method_new[i].iloc[0]['Test_Distance'])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


Labels = Analysis_similarity_Method_Label_App1_Levenshtein
Values_method = Analysis_similarity_Method_Method_App1_Levenshtein
Values_test = Analysis_similarity_Test_Test_App1_Levenshtein


angles=np.linspace(0,2*np.pi,len(Labels), endpoint=False)

angles=np.concatenate((angles,[angles[0]]))
Labels.append(Labels[0])
Values_method.append(Values_method[0])
Values_test.append(Values_test[0])

fig=plt.figure(figsize=(10,10),facecolor='#ffffff') #dadada
ax=fig.add_subplot(111, polar=True)

#First Plot
ax.plot(angles,Values_method, 'o-', color='green', linewidth=1, label='Methods')
ax.fill(angles, Values_method, alpha=0.25, color='green')

#Second Plot
ax.plot(angles,Values_test, 'o-', color='orange', linewidth=1, label='Test Cases')
ax.fill(angles, Values_test, alpha=0.25, color='orange')

#Labels

ax.set_thetagrids(angles * 180/np.pi, []) #Labels)
ax.set_rlabel_position(0)


plt.yticks([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1], ['0.1','0.2','0.3','0.4','0.5','0.6','0.7','0.8','0.9'], size=10)
plt.ylim(0,1)

plt.grid(True)
plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
print("MM - Approach 1:")
print("Count:", len(Values_method))
print("Mean: ", statistics.mean(Values_method))
print("Std:  ", statistics.stdev(Values_method))
print("Max:  ", max(Values_method))
print("Min:  ", min(Values_method))

In [ ]:
print("TT - Approach 1:")
print("Count:", len(Values_test))
print("Mean: ", statistics.mean(Values_test))
print("Std:  ", statistics.stdev(Values_test))
print("Max:  ", max(Values_test))
print("Min:  ", min(Values_test))

In [ ]:
1 - (len([ x for x in Values_test if x > 0.5 ]) / len(Values_test))

In [ ]:
1 - (len([ x for x in Values_test if x > 0.7 ]) / len(Values_test))

In [ ]:
sns.set(style="darkgrid")

sns.set(rc={'figure.figsize':(6,4)})

sns.histplot(data=Values_test, legend=False, color="orange", kde=True, bins=20)
plt.axvline(x=statistics.mean(Values_test),
            color='orange',
            ls='--',
            lw=1.75)

plt.legend()
plt.show()

In [ ]:
Analysis_similarity_TESTs_App1_Levenshtein = []
Analysis_similarity_App1_length_oracle = []
Analysis_similarity_App1_length_recom = []

for i in range(len(all_similar_method_new)):
    all_similar_method_new[i].sort_values(by=['Test_Method_Distance'], ascending=False, inplace=True)
    Analysis_similarity_TESTs_App1_Levenshtein.append(int(all_similar_method_new[i].iloc[0]['Test_Levenshtein']))
    Analysis_similarity_App1_length_oracle.append(len(tests[similar_methods_index_removed_duplicates[i]]))
    Analysis_similarity_App1_length_recom.append(len(tests[int(all_similar_method_new[i].iloc[0]['Index'])]))


In [ ]:
sns.set(style="darkgrid")

sns.set(rc={'figure.figsize':(6,4)})

sns.histplot(data=Analysis_similarity_TESTs_App1_Levenshtein, legend=False, color="orange", kde=True, bins=20)
plt.axvline(x=statistics.mean(Analysis_similarity_TESTs_App1_Levenshtein),
            color='orange',
            ls='--',
            lw=1.75)

#plt.legend()
plt.show()

In [ ]:
print("Evaluation Approach 1:")
print("Count:", len(Analysis_similarity_TESTs_App1_Levenshtein))
print("Mean: ", statistics.mean(Analysis_similarity_TESTs_App1_Levenshtein))
print("Std:  ", statistics.stdev(Analysis_similarity_TESTs_App1_Levenshtein))
print("Max:  ", max(Analysis_similarity_TESTs_App1_Levenshtein))
print("Min:  ", min(Analysis_similarity_TESTs_App1_Levenshtein))

In [ ]:
sns.set(style="darkgrid")

sns.set(rc={'figure.figsize':(6,4)})

sns.histplot(data=Analysis_similarity_App1_length_oracle, legend=False, color="orange", kde=True, bins=20)
plt.axvline(x=statistics.mean(Analysis_similarity_App1_length_oracle),
            color='orange',
            ls='--',
            lw=1.75)

#plt.legend()
plt.show()

In [ ]:
print("Evaluation Approach 1:")
print("Count:", len(Analysis_similarity_App1_length_oracle))
print("Mean: ", statistics.mean(Analysis_similarity_App1_length_oracle))
print("Std:  ", statistics.stdev(Analysis_similarity_App1_length_oracle))
print("Max:  ", max(Analysis_similarity_App1_length_oracle))
print("Min:  ", min(Analysis_similarity_App1_length_oracle))

In [ ]:
# merging two above histograms:

In [ ]:
sns.set(style="darkgrid")

sns.set(rc={'figure.figsize':(6,4)})

sns.histplot(data=Analysis_similarity_TESTs_App1_Levenshtein, legend=False, color="blue", kde=True, bins=20)
plt.axvline(x=statistics.mean(Analysis_similarity_TESTs_App1_Levenshtein),
            color='blue',
            ls='--',
            lw=1.75)
sns.histplot(data=Analysis_similarity_App1_length_oracle, legend=False, color="orange", kde=True, bins=20)
plt.axvline(x=statistics.mean(Analysis_similarity_App1_length_oracle),
            color='orange',
            ls='--',
            lw=1.75)

#plt.legend()
plt.show()

In [ ]:
sns.set(style="darkgrid")

sns.set(rc={'figure.figsize':(6,4)})

sns.histplot(data=Analysis_similarity_App1_length_recom, legend=False, color="orange", kde=True, bins=20)
plt.axvline(x=statistics.mean(Analysis_similarity_App1_length_recom),
            color='orange',
            ls='--',
            lw=1.75)

#plt.legend()
plt.show()

In [ ]:
print("Evaluation Approach 1:")
print("Count:", len(Analysis_similarity_App1_length_recom))
print("Mean: ", statistics.mean(Analysis_similarity_App1_length_recom))
print("Std:  ", statistics.stdev(Analysis_similarity_App1_length_recom))
print("Max:  ", max(Analysis_similarity_App1_length_recom))
print("Min:  ", min(Analysis_similarity_App1_length_recom))

### 10- Second approach evaluation

In [ ]:
Analysis_similarity_Method_and_Test_Label_App2_Levenshtein = []
Analysis_similarity_Method_Test_App2_Levenshtein = []
Analysis_similarity_Test_Test_App2_Levenshtein = []

for i in range(len(all_similar_method_and_test_new)):
    if (int(similar_methods_and_test_index_removed_duplicates[i]) != int(all_similar_method_and_test_new[i].iloc[0]['Index'])):
        all_similar_method_and_test_new[i].sort_values(by=['Method_Method_Levenshtein'], ascending=True, inplace=True)
        Analysis_similarity_Method_and_Test_Label_App2_Levenshtein.append((int(similar_methods_and_test_index_removed_duplicates[i]),int(all_similar_method_and_test_new[i].iloc[0]['Index'])))
        Analysis_similarity_Method_Test_App2_Levenshtein.append(all_similar_method_and_test_new[i].iloc[0]['Method_and_Tests_Distance'])
        Analysis_similarity_Test_Test_App2_Levenshtein.append(all_similar_method_and_test_new[i].iloc[0]['Test_Distance'])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


Labels = Analysis_similarity_Method_and_Test_Label_App2_Levenshtein
Values_method = Analysis_similarity_Method_Test_App2_Levenshtein
Values_test = Analysis_similarity_Test_Test_App2_Levenshtein


angles=np.linspace(0,2*np.pi,len(Labels), endpoint=False)

angles=np.concatenate((angles,[angles[0]]))
Labels.append(Labels[0])
Values_method.append(Values_method[0])
Values_test.append(Values_test[0])

fig=plt.figure(figsize=(10,10),facecolor='#ffffff') #dadada
ax=fig.add_subplot(111, polar=True)

#First Plot
ax.plot(angles,Values_method, 'o-', color='green', linewidth=1, label='Methods')
ax.fill(angles, Values_method, alpha=0.25, color='green')

#Second Plot
ax.plot(angles,Values_test, 'o-', color='orange', linewidth=1, label='Test Cases')
ax.fill(angles, Values_test, alpha=0.25, color='orange')

#Labels

ax.set_thetagrids(angles * 180/np.pi, []) #Labels)
ax.set_rlabel_position(0)


plt.yticks([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1], ['0.1','0.2','0.3','0.4','0.5','0.6','0.7','0.8','0.9'], size=10)
plt.ylim(0,1)

plt.grid(True)
plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
print("MM - Approach 2:")
print("Count:", len(Values_method))
print("Mean: ", statistics.mean(Values_method))
print("Std:  ", statistics.stdev(Values_method))
print("Max:  ", max(Values_method))
print("Min:  ", min(Values_method))

In [ ]:
print("TT - Approach 2:")
print("Count:", len(Values_test))
print("Mean: ", statistics.mean(Values_test))
print("Std:  ", statistics.stdev(Values_test))
print("Max:  ", max(Values_test))
print("Min:  ", min(Values_test))

In [ ]:
1 - (len([ x for x in Values_test if x > 0.5 ]) / len(Values_test))

In [ ]:
1 - (len([ x for x in Values_test if x > 0.7 ]) / len(Values_test))

In [ ]:
sns.set(style="darkgrid")

sns.set(rc={'figure.figsize':(6,4)})

sns.histplot(data=Values_test, legend=False, color="orange", kde=True, bins=20)
plt.axvline(x=statistics.mean(Values_test),
            color='orange',
            ls='--',
            lw=1.75)

plt.legend()
plt.show()

In [ ]:
Analysis_similarity_TESTs_App2_Levenshtein = []
Analysis_similarity_App2_length_oracle = []
Analysis_similarity_App2_length_recom = []

for i in range(len(all_similar_method_and_test_new)):
    if (int(similar_methods_and_test_index_removed_duplicates[i]) != int(all_similar_method_and_test_new[i].iloc[0]['Index'])):
        all_similar_method_and_test_new[i].sort_values(by=['Method_Method_Levenshtein'], ascending=True, inplace=True)
        Analysis_similarity_TESTs_App2_Levenshtein.append(int(all_similar_method_and_test_new[i].iloc[0]['Test_Levenshtein']))
        Analysis_similarity_App2_length_oracle.append(len(tests[similar_methods_and_test_index_removed_duplicates[i]]))
        Analysis_similarity_App2_length_recom.append(len(tests[int(all_similar_method_and_test_new[i].iloc[0]['Index'])]))


In [ ]:
sns.set(style="darkgrid")

sns.set(rc={'figure.figsize':(6,4)})

sns.histplot(data=Analysis_similarity_TESTs_App2_Levenshtein, legend=False, color="orange", kde=True, bins=20)
plt.axvline(x=statistics.mean(Analysis_similarity_TESTs_App2_Levenshtein),
            color='orange',
            ls='--',
            lw=1.75)

#plt.legend()
plt.show()

In [ ]:
print("Evaluation Approach 2:")
print("Count:", len(Analysis_similarity_TESTs_App2_Levenshtein))
print("Mean: ", statistics.mean(Analysis_similarity_TESTs_App2_Levenshtein))
print("Std:  ", statistics.stdev(Analysis_similarity_TESTs_App2_Levenshtein))
print("Max:  ", max(Analysis_similarity_TESTs_App2_Levenshtein))
print("Min:  ", min(Analysis_similarity_TESTs_App2_Levenshtein))

In [ ]:
sns.set(style="darkgrid")

sns.set(rc={'figure.figsize':(6,4)})

sns.histplot(data=Analysis_similarity_App2_length_oracle, legend=False, color="orange", kde=True, bins=20)
plt.axvline(x=statistics.mean(Analysis_similarity_App2_length_oracle),
            color='orange',
            ls='--',
            lw=1.75)

#plt.legend()
plt.show()

In [ ]:
print("Evaluation Approach 2:")
print("Count:", len(Analysis_similarity_App2_length_oracle))
print("Mean: ", statistics.mean(Analysis_similarity_App2_length_oracle))
print("Std:  ", statistics.stdev(Analysis_similarity_App2_length_oracle))
print("Max:  ", max(Analysis_similarity_App2_length_oracle))
print("Min:  ", min(Analysis_similarity_App2_length_oracle))

In [ ]:
sns.set(style="darkgrid")

sns.set(rc={'figure.figsize':(6,4)})

sns.histplot(data=Analysis_similarity_App2_length_recom, legend=False, color="orange", kde=True, bins=20)
plt.axvline(x=statistics.mean(Analysis_similarity_App2_length_recom),
            color='orange',
            ls='--',
            lw=1.75)

#plt.legend()
plt.show()

In [ ]:
print("Evaluation Approach 2:")
print("Count:", len(Analysis_similarity_App2_length_recom))
print("Mean: ", statistics.mean(Analysis_similarity_App2_length_recom))
print("Std:  ", statistics.stdev(Analysis_similarity_App2_length_recom))
print("Max:  ", max(Analysis_similarity_App2_length_recom))
print("Min:  ", min(Analysis_similarity_App2_length_recom))